
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning">
</div>


# Lab - Creating Bronze Tables from CSV Files
### Duration: ~15-20 minutes

This lab is divided into two sections: the **course lab** and an **optional challenge**. 

In a live class, if you finish the main lab early, feel free to attempt the challenge. You can also complete the challenge after class. 

**NOTE:** The challenge will require you to use resources outside the scope of this course.

### Learning Objectives
  - Inspect CSV files.
  - Read in CSV files as a Delta table and append with metadata columns.
  - Record malformed data types using the `_rescued_data` column.  

### Challenge Learning Objectives:
  - Clean malformed data types during ingestion by leveraging the `_rescued_data` column.

## REQUIRED - SELECT CLASSIC COMPUTE

Before executing cells in this notebook, please select your classic compute cluster in the lab. Be aware that **Serverless** is enabled by default and you have a Shared SQL warehouse.

<!-- ![Select Cluster](./Includes/images/selecting_cluster_info.png) -->

Follow these steps to select the classic compute cluster:


1. Navigate to the top-right of this notebook and click the drop-down menu to select your cluster. By default, the notebook will use **Serverless**.

2. If your cluster is available, select it and continue to the next cell. If the cluster is not shown:

   - Click **More** in the drop-down.

   - In the **Attach to an existing compute resource** window, use the first drop-down to select your unique cluster.

**NOTE:** If your cluster has terminated, you might need to restart it in order to select it. To do this:

1. Right-click on **Compute** in the left navigation pane and select *Open in new tab*.

2. Find the triangle icon to the right of your compute cluster name and click it.

3. Wait a few minutes for the cluster to start.

4. Once the cluster is running, complete the steps above to select your cluster.

# Course Lab - Creating Bronze Tables from CSV Files

### Scenario

You are working with your data team on ingesting a CSV file into Databricks. However, you notice there is a malformed row in your CSV file. Your job is to ingest the file and rescue the malformed data and write as a Delta table.


## A. Classroom Setup

Run the following cell to configure your working environment for this notebook.

**NOTE:** The `DA` object is only used in Databricks Academy courses and is not available outside of these courses. It will dynamically reference the information needed to run the course in the lab environment.

In [0]:
%run ./Includes/Classroom-Setup-05L

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


----------------------------------------------------------------------------------------
Directory /Volumes/dbacademy/ops/labuser11045124_1753930326@vocareum_com/csv_demo_files already exists. No action taken.
Directory /Volumes/dbacademy/ops/labuser11045124_1753930326@vocareum_com/json_demo_files already exists. No action taken.
Directory /Volumes/dbacademy/ops/labuser11045124_1753930326@vocareum_com/xml_demo_files already exists. No action taken.
----------------------------------------------------------------------------------------



----------------------------------------------------------------------------------------
Directory /Volumes/dbacademy/ops/labuser11045124_1753930326@vocareum_com/csv_demo_files already exists. No action taken.
Directory /Volumes/dbacademy/ops/labuser11045124_1753930326@vocareum_com/json_demo_files already exists. No action taken.
Directory /Volumes/dbacademy/ops/labuser11045124_1753930326@vocareum_com/xml_demo_files already exists. No action taken.
----------------------------------------------------------------------------------------



DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

Run the cell below to view your default catalog and schema. Notice that your default catalog is **dbacademy** and your default schema is your unique **labuser** schema.

**NOTE:** The default catalog and schema are pre-configured for you to avoid the need to specify the three-level name when writing your tables (i.e., catalog.schema.table).

In [0]:
SELECT current_catalog(), current_schema()

current_catalog(),current_schema()
dbacademy,labuser11045124_1753930326


## B. Lab - CSV File Ingestion
Ingest the CSV file as a Delta table.

### B1. Inspect the Dataset

1. In the cell below, view the value of the SQL variable `DA.paths_working_dir`. This variable will reference your **labuser** volume, as each user has a different source volume. This variable is created within the classroom setup script to dynamically reference your unique volume.

   Run the cell and review the results. You’ll notice that the `DA.paths_working_dir` variable points to your `/Volumes/dbacademy/ops/labuser` volume.

**Note:** Instead of using the `DA.paths_working_dir` variable, you could also specify the path name directly by right clicking on your volume and selecting **Copy volume path**.

In [0]:
values(DA.paths_working_dir)

col1
/Volumes/dbacademy/ops/labuser11045124_1753930326@vocareum_com


2. You can concatenate the `DA.paths_working_dir` SQL variable with a string to specify a specific subdirectory in your volume.

   Run the cell below and review the results. You’ll notice that it returns the path to your **lab_malformed_data.csv** file. This method will be used when referencing your volume within the `read_files` function.

In [0]:
values(DA.paths_working_dir || '/csv_demo_files/lab_malformed_data.csv')

col1
/Volumes/dbacademy/ops/labuser11045124_1753930326@vocareum_com/csv_demo_files/lab_malformed_data.csv


3. Next, let's take a look at our CSV file. 

    Copy the path from the output below and paste it within the backticks in the query to reference the **lab_malformed_data.csv** file.

    The query should use `text.<path_from_above>` to return the headers and rows from the CSV file. 

    Run the cell and view row 4. Notice that the value for the price contains a `$`.

In [0]:
SELECT *
FROM text.`/Volumes/dbacademy/ops/labuser11045124_1753930326@vocareum_com/csv_demo_files/lab_malformed_data.csv`;

value
"item_id,name,price"
"M_PREM_Q,Premium Queen Mattress,1795.0"
"M_STAN_F,Standard Full Mattress,945.0"
"M_PREM_A,Premium Queen Mattress,$100.00"


### B2. Ingesting and Rescuing Malformed Data

Begin developing your query to ingest the CSV file in the specified path and view malformed records using the **_rescued_data** column.

#### Requirements
Your final SQL query should ingest the CSV file using CTAS and `read_files`. **In the cell below, do not create a table yet. Simply start developing your query to ingest and create the table**:

1. Select all columns from the raw CSV file.

2. Use the `read_files()` function with appropriate options to read the CSV file. 
   - **HINT:** Note that the delimiter is a comma (`,`) not a pipe (`|`).

3. Explicitly define the schema for ingestion. The schema is defined as follows:  
   - `item_id` (STRING)  
   - `name` (STRING)  
   - `price` (DOUBLE)

4. Use the correct option to include the rescued data column and name it **_rescued_data** to capture malformed rows.

   - **HINT**: If you define a schema you must [use the rescuedDataColumn option](https://docs.databricks.com/aws/en/sql/language-manual/functions/read_files#csv-options) to add the **_rescued_data** column.

**SOLUTION OUTPUT**

Your output result should look like the following:
| item_id (STRING)   | name (STRING)                   | price (DOUBLE)| _rescued_data (STRING)                                                                                         |
|-----------|-------------------------|-------|-----------------------------------------------------------------------------------------------------------|
| M_PREM_Q  | Premium Queen Mattress   | 1795  | null                                                                                                      |
| M_STAN_F  | Standard Full Mattress   | 945   | null                                                                                                      |
| M_PREM_A  | Premium Queen Mattress   | null  | {"price":"$100.00","_file_path":"dbfs:/Volumes/dbacademy/ops/peter_s@databricks_com/csv_demo_files/lab_malformed_data.csv"} |


In [0]:
SELECT item_id, name, price, _rescued_data, _metadata.file_modification_time AS file_modification_time, _metadata.file_name AS source_file, current_timestamp() as ingestion_timestamp
FROM read_files(
        DA.paths_working_dir || '/csv_demo_files/lab_malformed_data.csv',
        format => "csv",
        sep => ",", 
        header => true, 
        schema => 'item_id STRING, name STRING, price DOUBLE', 
        rescueddatacolumn => "_rescued_data"
      );

item_id,name,price,_rescued_data,file_modification_time,source_file,ingestion_timestamp
M_PREM_Q,Premium Queen Mattress,1795.0,null,2025-07-31T03:15:05Z,lab_malformed_data.csv,2025-07-31T03:21:47.975211Z
M_STAN_F,Standard Full Mattress,945.0,null,2025-07-31T03:15:05Z,lab_malformed_data.csv,2025-07-31T03:21:47.975211Z
M_PREM_A,Premium Queen Mattress,null,"{""price"":""$100.00"",""_file_path"":""dbfs:/Volumes/dbacademy/ops/labuser11045124_1753930326@vocareum_com/csv_demo_files/lab_malformed_data.csv""}",2025-07-31T03:15:05Z,lab_malformed_data.csv,2025-07-31T03:21:47.975211Z


### B3. Add Additional Metadata Columns During Ingestion

Next, you can create the final bronze table named **05_lab_bronze** that contains the additional metadata columns. Use the query you created above as the starting point.

### Final Table Requirements

Incorporate the SQL query you created in the previous section and complete the following:

1. Use a CTAS statement to create the final bronze Delta table named **05_lab_bronze**. 

1. Ingest the same CSV file `/Volumes/dbacademy/ops/<username>/csv_demo_files/lab_malformed_data.csv`

1. Use the same defined schema:  
   - `item_id` (STRING)  
   - `name` (STRING)  
   - `price` (DOUBLE)

1. Use the `_metadata` column to create two new columns named **file_modification_time** and **source_file**  within your SELECT statement.
   - **HINT:** [_metadata](https://docs.databricks.com/en/ingestion/file-metadata-column.html)

1. Add a column named **ingestion_time** that provides a timestamp for ingestion. 
   - **HINT:** Use the [current_timestamp()](https://docs.databricks.com/aws/en/sql/language-manual/functions/current_timestamp) to record the current timestamp at the start of the query evaluation. 


**NOTE:** If you need to see how your final table should look, run the next cell to view the solution table **05_lab_solution**.

In [0]:
-- Run if you want to see the final table
SELECT * 
FROM 05_lab_solution

item_id,name,price,_rescued_data,file_modification_time,source_file,ingestion_time
M_PREM_Q,Premium Queen Mattress,1795.0,null,2025-07-31T03:15:05Z,lab_malformed_data.csv,2025-07-31T03:15:06.158776Z
M_STAN_F,Standard Full Mattress,945.0,null,2025-07-31T03:15:05Z,lab_malformed_data.csv,2025-07-31T03:15:06.158776Z
M_PREM_A,Premium Queen Mattress,null,"{""price"":""$100.00"",""_file_path"":""dbfs:/Volumes/dbacademy/ops/labuser11045124_1753930326@vocareum_com/csv_demo_files/lab_malformed_data.csv""}",2025-07-31T03:15:05Z,lab_malformed_data.csv,2025-07-31T03:15:06.158776Z


In [0]:
---- Drop the table if it exists for demonstration purposes
DROP TABLE IF EXISTS 05_lab_bronze;

---- Complete your CTAS statement below:
CREATE TABLE 05_lab_bronze AS
SELECT
  item_id,
  name,
  price,  
  _rescued_data,
  file_modification_time,
  source_file,
  ingestion_timestamp
FROM (
  SELECT
    item_id,
    name,
    price,
    _rescued_data,
    _metadata.file_modification_time AS file_modification_time,
    _metadata.file_name AS source_file,
    current_timestamp() as ingestion_timestamp
  FROM read_files(
          CONCAT(DA.paths_working_dir, '/csv_demo_files/lab_malformed_data.csv'),
          format => "csv",
          sep => ",",   
          header => true, 
          schema => 'item_id STRING, name STRING, price DOUBLE', 
          rescueddatacolumn => "_rescued_data"
        )
) t
WHERE item_id IS NOT NULL OR _rescued_data IS NOT NULL;

org.apache.spark.SparkException: [FAILED_READ_FILE.NO_HINT] Error while reading file dbfs:/Volumes/dbacademy/ops/labuser11045124_1753930326@vocareum_com/csv_demo_files/lab_malformed_data.csv.  SQLSTATE: KD001
	at org.apache.spark.sql.errors.QueryExecutionErrors$.cannotReadFilesError(QueryExecutionErrors.scala:1151)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.logErrorFileNameAndThrow(FileScanRDD.scala:865)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.getNext(FileScanRDD.scala:820)
	at org.apache.spark.util.NextIterator.hasNext(NextIterator.scala:73)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:982)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.$anonfun$hasNext$1(FileScanRDD.scala:642)
	at scala.runtime.java8.JFunction0$mcZ$sp.apply(JFunction0$mcZ$sp.java:23)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at 

Run the cell below to view your final table **05_lab_bronze** and compare it with the solution table.

In [0]:
-- View the final table
SELECT * 
FROM 05_lab_bronze

item_id,name,price,_rescued_data,file_modification_time,source_file,ingestion_timestamp
M_PREM_Q,Premium Queen Mattress,1795.0,null,2025-07-31T03:15:05Z,lab_malformed_data.csv,2025-07-31T03:23:56.993616Z
M_STAN_F,Standard Full Mattress,945.0,null,2025-07-31T03:15:05Z,lab_malformed_data.csv,2025-07-31T03:23:56.993616Z
M_PREM_A,Premium Queen Mattress,null,"{""price"":""$100.00"",""_file_path"":""dbfs:/Volumes/dbacademy/ops/labuser11045124_1753930326@vocareum_com/csv_demo_files/lab_malformed_data.csv""}",2025-07-31T03:15:05Z,lab_malformed_data.csv,2025-07-31T03:23:56.993616Z


## C. (Optional) Challenge: Rescuing Data
The challenge is optional during a live class if you have time. This challenge may require you to use documentation. If you run out of time during a live class, try and complete this after class.

#### Optional Challenge Scenario
You report back to your data team and everyone agrees to clean up any values in the rescued data column that contain a `$` when ingesting as a bronze table. To fix this issue, you agree to handle these edge case during ingestion by leveraging the `_rescued_data` column.

Your team has decided to strip the `$` in the price and simply store the numeric value during ingestion.


#### Requirements

- Complete the SQL query below to modify the **_rescued_data** column and correctly fixes the malformed value.

- Your final table should return the following columns:
  - **item_id**
  - **name**
  - **price** (the original price column)
  - **TO DO**: **price_fixed** (fix the malformed row in the rescued data column `$100` and return all prices a numeric values)
  - **_rescued_data**
  - **source_file**
  - **file_modification_time**
  - **ingestion_timestamp**

- Use `CREATE TABLE AS` with `read_files()` to read the CSV file and create a table named .

**HINT:** One solution you can use is the [`COALESCE()`](https://docs.databricks.com/aws/en/sql/language-manual/functions/coalesce) function along with the [`REPLACE()`](https://docs.databricks.com/aws/en/sql/language-manual/functions/replace) function to replace the malformed string price (`$100`) with `100`.

You can view the final table by running the next cell if you would like a hint for building your query.

In [0]:
-- Run if you want to see the final table
SELECT * 
FROM 05_lab_challenge_solution;

item_id,name,price,price_fixed,_rescued_data,file_modification_time,source_file,ingestion_time
M_PREM_Q,Premium Queen Mattress,1795.0,1795.0,null,2025-07-31T03:15:05Z,lab_malformed_data.csv,2025-07-31T03:15:09.128026Z
M_STAN_F,Standard Full Mattress,945.0,945.0,null,2025-07-31T03:15:05Z,lab_malformed_data.csv,2025-07-31T03:15:09.128026Z
M_PREM_A,Premium Queen Mattress,null,100.00,"{""price"":""$100.00"",""_file_path"":""dbfs:/Volumes/dbacademy/ops/labuser11045124_1753930326@vocareum_com/csv_demo_files/lab_malformed_data.csv""}",2025-07-31T03:15:05Z,lab_malformed_data.csv,2025-07-31T03:15:09.128026Z


In [0]:
CREATE TABLE 05_lab_challenge
SELECT
  item_id,
  name,
  price,
  COALESCE(price, CAST(REPLACE(_rescued_data, '$', '') AS DOUBLE)) AS price_fixed,   -- CLEAN the rescued data column and return it as a numeric value with the other prices
  _rescued_data,
  _metadata.file_modification_time AS file_modification_time,
  _metadata.file_name AS source_file, 
  current_timestamp() as ingestion_timestamp
FROM read_files(
        DA.paths_working_dir || '/csv_demo_files/lab_malformed_data.csv',
        format => "csv",
        sep => ",",
        header => true,
        schema => 'item_id STRING, name STRING, price DOUBLE', 
        rescueddatacolumn => "_rescued_data"
      );

num_affected_rows,num_inserted_rows



&copy; 2025 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="blank">Apache Software Foundation</a>.<br/>
<br/><a href="https://databricks.com/privacy-policy" target="blank">Privacy Policy</a> | 
<a href="https://databricks.com/terms-of-use" target="blank">Terms of Use</a> | 
<a href="https://help.databricks.com/" target="blank">Support</a>